In [42]:
import time
import pandas as pd
from tqdm.notebook import tqdm  # <- For Jupyter Notebook
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog, commonplayerinfo

# Get all NBA players
all_players = players.get_players()
print(f"Found {len(all_players)} players.")

# Prepare master DataFrame
master_df = pd.DataFrame()

# Use tqdm to wrap the players list
for player in tqdm(all_players, desc="Players Progress", unit="player"):
    player_id = player['id']
    player_name = player['full_name']
    
    try:
        # Get FROM_YEAR and TO_YEAR
        info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
        data = info.get_data_frames()[0]
        from_year = int(data['FROM_YEAR'].values[0])
        to_year = int(data['TO_YEAR'].values[0])
        
        for year in range(from_year, to_year + 1):
            season_start = str(year)
            season_end = str((year + 1) % 100).zfill(2)
            season = f"{season_start}-{season_end}"
            
            try:
                # Pull game logs for this season
                gamelog = playergamelog.PlayerGameLog(player_id=player_id, season=season, season_type_all_star='Regular Season')
                df = gamelog.get_data_frames()[0]
                
                if not df.empty:
                    df['PLAYER_NAME'] = player_name
                    df['SEASON'] = season
                    master_df = pd.concat([master_df, df], ignore_index=True)
                    
                time.sleep(0.7)  # Sleep to respect API limits
                
            except Exception as e:
                print(f"  Error pulling {player_name} {season}: {e}")
                time.sleep(1)

    except Exception as e:
        print(f"Error getting info for {player_name}: {e}")
        time.sleep(1)

# Save to CSV
master_df.to_csv('nba_player_game_logs_alltime.csv', index=False)
print("✅ Done! Saved all player game logs.")

Found 5024 players.


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html